In [1]:
import numpy as np
import sigpy as sp
import sigpy.plot as pl
from skimage.metrics import structural_similarity as np_ssim
import vd_spiral
import sigpy.mri
import torch
import os
import sys
sys.path.append('/mnt/mnt/5TB_slot2/Tobias')
sys.path.append('/mnt/mnt/5TB_slot2/Tobias/TobiasPy/fastMRI')
sys.path.append('/mnt/mnt/4TB_pcie/fastBrain/')
import h5py
import net_utils as nu
import argparse
from common.args import Args
import pathlib
import random
from torch.utils.data import Dataset
import time
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
sys.argv=['']
def create_arg_parser():
    #timestamp = calendar.timegm(time.gmtime())
    #human_readable = datetime.datetime.fromtimestamp(timestamp).isoformat()
    #human_readable="pure_inverse_fourier"
    human_readable="Compressed_Sensing_Comparison"
    parser = argparse.ArgumentParser(description="ML parameters")
    parser.add_argument('--num-pools', type=int, default=4, help='Number of U-Net pooling layers')
    parser.add_argument('--drop-prob', type=float, default=0.0, help='Dropout probability')
    parser.add_argument('--num-chans', type=int, default=32, help='Number of U-Net channels')

    parser.add_argument('--batch-size', default=1, type=int, help='Mini batch size')
    parser.add_argument('--num-epochs', type=int, default=250, help='Number of training epochs')
    parser.add_argument('--lr', type=float, default=0.0001, help='Learning rate')
    parser.add_argument('--lr-step-size', type=int, default=100,
                        help='Period of learning rate decay')
    parser.add_argument('--lr-gamma', type=float, default=0.1,
                        help='Multiplicative factor of learning rate decay')
    parser.add_argument('--weight-decay', type=float, default=0.01,
                        help='Strength of weight decay regularization')
    parser.add_argument('--momentum', type=float, default=0.1,
                        help='Strength of optimizer momentum')

    parser.add_argument('--report-interval', type=int, default=10000, help='Period of loss reporting')
    parser.add_argument('--data-parallel', default=True,
                        help='If set, use multiple GPUs using data parallelism')
    parser.add_argument('--device', type=str, default='cuda',
                        help='Which device to train on. Set to "cuda" to use the GPU')
    parser.add_argument('--exp-dir', type=pathlib.Path, default='/mnt/mnt/5TB_slot2/Tobias/Thesis/FF_lrelu_'+str(human_readable),
                        help='Path where model and results should be saved')
    parser.add_argument('--resume', action='store_true', default=False,
                        help='If set, resume the training from a previous model checkpoint. '
                             '"--checkpoint" should be set with this')
    parser.add_argument('--checkpoint', type=str, default='/mnt/mnt/5TB_slot2/Tobias/Thesis/FF_lrelu_'+str(human_readable)+'/best_model.pt',
                        help='Path to an existing checkpoint. Used along with "--resume"')
    parser.add_argument('--logdir', type=str, default='/mnt/mnt/5TB_slot2/Tobias/Thesis/log/ff_lrelu_'+str(human_readable),
                        help='Path to an existing checkpoint. Used along with "--resume"')
    parser.add_argument('--seed', default=42, type=int, help='Seed for random number generators')
    parser.add_argument('--resolution', default=128, type=int, help='Resolution of images')
    parser.add_argument('--device_ids', default=[0,1] , help='GPUS used')
    return parser
args=create_arg_parser().parse_args()
print("save to",args.checkpoint)

save to /mnt/mnt/5TB_slot2/Tobias/Thesis/FF_lrelu_Compressed_Sensing_Comparison/best_model.pt


In [3]:
class SliceData(Dataset):
    """
    A PyTorch Dataset that provides access to MR image slices.
    """
    
    def __init__(self, root="/mnt/mnt/5TB_slot2/fastMRI/multicoil_train", sample_rate=1.):
        self.examples = []
        #potential to apply pre-process transform
        files = list(pathlib.Path(root).iterdir())
        if sample_rate < 1:
            #random.shuffle(files)
            num_files = round(len(files) * sample_rate)
            files = files[:num_files]
        for fname in sorted(files):
            print(fname)
            try:
                #print(h5py.File(fname, 'r').keys())
                #maybe also only randomly select parts of slices. Otherwise lot of outer noise-like structures.
                xml = h5py.File(fname, 'r')['ismrmrd_header']
                kspace = h5py.File(fname, 'r')['kspace']
                num_slices = kspace.shape[0]
                num_channels=kspace.shape[1]
                if (kspace.shape[-2]<args.resolution) or (kspace.shape[-1]<args.resolution) or num_channels!=16:
                    continue
                print(len(self.examples))
                for slice in range(num_slices):
                    crop_size = (min(args.resolution, kspace.shape[-2]),min(args.resolution, kspace.shape[-1]))
                    k=nu.center_crop(kspace[slice], crop_size)
                    target=abs(nu.rsos(nu.make_ift(k),0))
                    m=np.max(target)
                    if m>0:
                        target/=m
                    else:
                        continue
                    k=nu.to_tensor(k)
                    print(k.shape)
                    us=nu.apply_mask(k,r=3,mode="mid")[0]#get only the data part of the data,mask double
                    print(us.shape)
                    us/=m
                    if np.isnan(np.sum(target)):
                        continue
                    X=us#.flatten()
                    print(X.shape)
                    #print(X.shape)
                    Y=nu.to_tensor(target)
                    #print(Y.shape)
                    #X,Y=test_fourier()
                    self.examples += [(fname, slice, X, Y)]
            except:
                print("Couldn't open file")
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, i):
        fname, slice, f, t = self.examples[i]
#         with h5py.File(fname, 'r') as data:
#             full=to_tensor(data['kspace'][slice])
#             crop_size = (min(args.resolution, full.shape[-3]),min(args.resolution, full.shape[-2]))          
#             full=complex_center_crop(full,crop_size)
            
#             kspace = to_tensor(data['kspace'][slice])
#             crop_size = (min(args.resolution, kspace.shape[-3]), min(args.resolution, kspace.shape[-2]))
#             kspace=apply_mask(kspace)[0]
#             kspace = complex_center_crop(kspace, crop_size)
            
        return (f, t)

In [4]:
def create_data_loaders():
    #batch_size=32
    #mask_func = None
    #if args.mask_kspace:
    #    mask_func = MaskFunc(args.center_fractions, args.accelerations)
    train_data = SliceData(root="/mnt/mnt/4TB_pcie/fastBrain/multicoil_train",sample_rate=1)
    dev_data = SliceData(root="/mnt/mnt/4TB_pcie/fastBrain/multicoil_test_brain",sample_rate=1)
    test_data = SliceData(root="/mnt/mnt/4TB_pcie/fastBrain/multicoil_test_brain",sample_rate=1)
    train_loader = DataLoader(
        dataset=train_data,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
    )
    dev_loader = DataLoader(
        dataset=dev_data,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
    )
    test_loader=None
#     test_loader = DataLoader(
#         dataset=test_data,
#         num_workers=0,
#         pin_memory=True,
#     )
    return train_loader, dev_loader, test_loader

In [ ]:
t=time.time()
train_loader, dev_loader, test_loader= create_data_loaders()
print("Time taken to load data: ",time.time()-t)

/mnt/mnt/4TB_pcie/fastBrain/multicoil_train/file_brain_AXFLAIR_200_6002425.h5
/mnt/mnt/4TB_pcie/fastBrain/multicoil_train/file_brain_AXFLAIR_200_6002428.h5
/mnt/mnt/4TB_pcie/fastBrain/multicoil_train/file_brain_AXFLAIR_200_6002429.h5
/mnt/mnt/4TB_pcie/fastBrain/multicoil_train/file_brain_AXFLAIR_200_6002435.h5
0
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 

torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size

torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
/mnt/mnt/4TB_pcie/fastBrain/multicoil_train/file_brain_AXFLAIR_200_6002501.h5
/mnt/mnt/4TB_pcie/fastBrain/multicoil_train/file_brain_AXFLAIR_200_6002503.h5
/mnt/mnt/4TB_pcie/fastBrain/multic

torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
/mnt/mnt/4TB_pcie/fastBrain/multicoil_train/file_brain_AXFLAIR_200_6002520.h5
256
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 12

torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size

...

torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
/mnt/mnt/4TB_pcie/fastBrain/multicoil_train/file_brain_AXFLAIR_201_6003012.h5
908
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 12

torch.Size([16, 128, 128, 2])
/mnt/mnt/4TB_pcie/fastBrain/multicoil_train/file_brain_AXFLAIR_202_6000424.h5
982
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 12

torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
/mnt/mnt/4

torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
/mnt/mnt/4TB_pcie/fastBrain/multicoil_train/file_brain_AXFLAIR_202_6000492.h5
1158
torch.Size([16, 1

torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
/mnt/mnt/4TB_pcie/fastBrain/multicoil_train/file_brain_AXFLAIR_202_6000511.h5
1238
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 128, 128, 2])
torch.Size([16, 1

In [ ]:
#test visualization

with torch.no_grad():
        for (f, t) in train_loader:
            print(f.shape,t.shape)
            plt.imshow(abs(nu.make_ift(nu.to_complex(f)))[0,0,:,:])
            plt.show()
            plt.imshow(t[0,:,:])
            plt.show()
            lamda = 0.01
            img_sense = sigpy.mri.app.SenseRecon(nu.to_complex(f)[0,:,:,:], mps, lamda=l).run()
            pl.ImagePlot(mps, z=0, title='Sensitivity Maps Estimated by ESPIRiT')
            f=nu.to_complex(f)[0,:,:,:]
            
            lamda = 0.01
            img_sense = sigpy.mri.app.SenseRecon(f, mps, lamda=l).run()
            #pl.ImagePlot(img_sense, title='SENSE Reconstruction')
            plt.imshow(abs(img_sense))
            plt.show()
            
            lamda = 0.005
            img_tv = sigpy.mri.app.TotalVariationRecon(f, mps, lamda=l).run()
            #pl.ImagePlot(y, title='Total Variation Regularized Reconstruction')
            plt.imshow(abs(img_tv))
            plt.show()
            
            l=0.005
            img_l1=sigpy.mri.app.L1WaveletRecon(f, mps, lamda=l).run()
            #pl.ImagePlot(img_l1, title='L1 Wavelet Regularized Reconstruction')
            plt.imshow(abs(img_l1))
            plt.show()
            break;